# Data Loading

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 100)

In [2]:
import os
os.listdir()

['V4.1 Last 600K with email.csv',
 '.DS_Store',
 'Contact+Account 250K-500K.xlsx',
 '4.4 After 4_14.csv',
 'Snowflake SQL Query.rtf',
 'Persona Matching V4.1 - Include email and more records 3.11.ipynb',
 'Persona Matching V4.3 - Adjust Head of IT Operations.ipynb',
 'V4.3 After 3_22.csv',
 'V4.1 First 650K with email.csv',
 'AssignedPersona.csv',
 'AllContactData.csv',
 'AssignedPersonaandTitleLevel Second 800K.csv',
 'AssignedPersonaandTitleLevel First 1M.csv',
 'Corporate.csv',
 '.ipynb_checkpoints',
 'Head of IT Titles.xlsx',
 'Persona Matching V4.4 - Adjust DevOps, DevSecOps, IT Security.ipynb',
 'V4.1 Second 650K with email.csv',
 'SpecialPersona.csv',
 'SMB.csv',
 'Enterprise.csv',
 'V4.2 After 3_11.csv',
 'Persona Matching V4.2 - Include email and new personas.ipynb']

In [3]:
#Import multiple files of data that were downloaded separately from Snowflake
df1 = pd.read_csv('V4.1 First 650K with email.csv')[0:650000]
df2 = pd.read_csv('V4.1 Second 650K with email.csv')[0:650000]

/Users/czhang/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#Consolidate all data into 1 file

Contact = pd.concat([df1,df2],ignore_index=True)
Contact.shape

(1985912, 11)

In [6]:
Contact.tail(5)

,ID,TITLE,JOB_FUNCTION__C,EMAIL,GEO_LOCATION_LEVEL1__C,REGION__C,OWNER_SEGMENT__C,PRIMARY_VERTICAL__C,REVENUE__C,EMPLOYEE_COUNT__C,TRANSLATED_TITLE
1985907,0036T00004AWSVCQA5,Chief Operating Officer,NaN,kpepper@nossaman.com,AMS,United States,NaN,NaN,NaN,NaN,Chief Operating Officer
1985908,0036T00004AWXs8QAH,Senior Network Engineer,NaN,sgilbreath@jacksonhealthcare.com,AMS,United States,NaN,NaN,NaN,NaN,Senior Network Engineer
1985909,0036T00004AUk9jQAD,Yusa IT Administrator,IT: Systems Admin,rob.juliano@yusa-oh.com,AMS,United States,NaN,NaN,NaN,NaN,Yusa IT Administrator
1985910,0036T00004AW28jQAD,Senior Software Developer,NaN,achakraborty@hibbertgroup.com,AMS,United States,NaN,NaN,NaN,NaN,Senior Software Developer
1985911,0036T00004AUNsRQAX,Computer Systems Administrator,IT: Systems Admin,NaN,AMS,United States,NaN,NaN,NaN,NaN,Computer Systems Administrator


# Variable Selection 

In [7]:
#choose useful variables for segmentation

useful = Contact[['ID','TRANSLATED_TITLE', 'JOB_FUNCTION__C', 'EMAIL', 'OWNER_SEGMENT__C',
                  'GEO_LOCATION_LEVEL1__C', 'REGION__C', 'EMPLOYEE_COUNT__C']]
useful.shape

(1985912, 8)

# Data Cleaning

In [8]:
#Some columns have so many 'other's, let us remove them

useful= useful.replace(['Other'],'')
useful.head(5)

,ID,TRANSLATED_TITLE,JOB_FUNCTION__C,EMAIL,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C
0,0036T0000474qgbQAA,Senior IT Manager,IT: Manager,brian.guevara@colorofchange.org,CORP,AMS,United States,11.0
1,0030W00003R8xlVQAR,CFO,NaN,ismael.savadogo@colorofchange.org,CORP,AMS,United States,11.0
2,0030W00003qpL0bQAE,Director of Operations,NaN,sue.nyoni@colorofchange.org,CORP,AMS,United States,11.0
3,0030W00003KBeetQAD,Chief of Staff,"Executive (CEO, CMO, COO, CFO)",keith@colorofchange.org,CORP,AMS,United States,11.0
4,0030W00003KBjJgQAL,AVP of Information Technology,IT: VP or Director,acarter@homesavings.com,CORP,AMS,United States,500.0


In [9]:
##lower the string columns

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

#keep string columns
ID = useful[["ID","EMAIL"]]
useful_string = useful[["TRANSLATED_TITLE", "JOB_FUNCTION__C", "OWNER_SEGMENT__C",
                   "GEO_LOCATION_LEVEL1__C", "REGION__C"]]

#convert all text to lower case for future NLP
useful_string = useful_string.applymap(lambda s: s.lower() if type(s) == str else s)

#separate numeric columns
useful_numeric = useful[["EMPLOYEE_COUNT__C"]]

#combine all columns
useful_lower= pd.concat([ID, useful_string, useful_numeric], axis=1, ignore_index=False)
useful_lower.head(5) 

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C
0,0036T0000474qgbQAA,brian.guevara@colorofchange.org,senior it manager,it: manager,corp,ams,united states,11.0
1,0030W00003R8xlVQAR,ismael.savadogo@colorofchange.org,cfo,NaN,corp,ams,united states,11.0
2,0030W00003qpL0bQAE,sue.nyoni@colorofchange.org,director of operations,NaN,corp,ams,united states,11.0
3,0030W00003KBeetQAD,keith@colorofchange.org,chief of staff,"executive (ceo, cmo, coo, cfo)",corp,ams,united states,11.0
4,0030W00003KBjJgQAL,acarter@homesavings.com,avp of information technology,it: vp or director,corp,ams,united states,500.0


In [10]:
#remove all floating NaN

useful_nona = useful_lower.fillna('')

In [11]:
useful_nona.shape

(1985912, 8)

# Remove Rows without any Title

In [12]:
RemoveEmptyTitle = useful_nona[useful_nona['TRANSLATED_TITLE'] != '']

In [13]:
RemoveEmptyTitle.shape

(1801530, 8)

In [14]:
NoTitle = useful_nona[useful_nona['TRANSLATED_TITLE'] == '']

In [15]:
NoTitle.shape

(184382, 8)

# Remove Alliances

In [16]:
RemoveAlliance = RemoveEmptyTitle[RemoveEmptyTitle['OWNER_SEGMENT__C'] != 'alliances']

In [17]:
RemoveAlliance.shape

(1718858, 8)

In [18]:
Alliance = RemoveEmptyTitle[RemoveEmptyTitle['OWNER_SEGMENT__C'] == 'alliances']

In [19]:
Alliance.shape

(82672, 8)

# Change Employee Count to Numeric

In [20]:
RemoveAlliance['EMPLOYEE_COUNT__C'] = pd.to_numeric(RemoveAlliance['EMPLOYEE_COUNT__C'])

/var/folders/7r/1qz24gm934z0mlg9dwwq2hpr0000gn/T/ipykernel_76112/2931606309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RemoveAlliance['EMPLOYEE_COUNT__C'] = pd.to_numeric(RemoveAlliance['EMPLOYEE_COUNT__C'])


# Assign Segment to Those without any Segment

In [21]:
NoSegment = RemoveAlliance.loc[RemoveAlliance['OWNER_SEGMENT__C'] == '']

In [22]:
NoSegment.shape

(122140, 8)

In [23]:
NoSegment['EMPLOYEE_COUNT__C'].dtype

dtype('float64')

In [24]:
import numpy as np

conditions = [(NoSegment['EMPLOYEE_COUNT__C'] <= 250),
    (NoSegment['EMPLOYEE_COUNT__C'] > 250) & (NoSegment['EMPLOYEE_COUNT__C'] < 2500),
    (NoSegment['EMPLOYEE_COUNT__C'] > 2501)]

values = ['smb', 'corp', 'ent']

NoSegment['OWNER_SEGMENT__C'] = np.select(conditions, values)

/var/folders/7r/1qz24gm934z0mlg9dwwq2hpr0000gn/T/ipykernel_76112/2596145105.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NoSegment['OWNER_SEGMENT__C'] = np.select(conditions, values)


In [25]:
#There are 3483 records without employee count, let us assume they belong to CORP, for the sake of clustering
NoSegment["OWNER_SEGMENT__C"].replace({"0": "corp"}, inplace=True)
NoSegment.tail(5)

/Users/czhang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C
1985907,0036T00004AWSVCQA5,kpepper@nossaman.com,chief operating officer,,corp,ams,united states,NaN
1985908,0036T00004AWXs8QAH,sgilbreath@jacksonhealthcare.com,senior network engineer,,corp,ams,united states,NaN
1985909,0036T00004AUk9jQAD,rob.juliano@yusa-oh.com,yusa it administrator,it: systems admin,corp,ams,united states,NaN
1985910,0036T00004AW28jQAD,achakraborty@hibbertgroup.com,senior software developer,,corp,ams,united states,NaN
1985911,0036T00004AUNsRQAX,,computer systems administrator,it: systems admin,corp,ams,united states,NaN


# Combine NoSegment to the Original Dataset with Segment

In [26]:
WithSegment = RemoveAlliance.loc[RemoveAlliance['OWNER_SEGMENT__C'] != '']
WithSegment.shape

(1596718, 8)

In [27]:
WithSegment

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C
0,0036T0000474qgbQAA,brian.guevara@colorofchange.org,senior it manager,it: manager,corp,ams,united states,11.0
1,0030W00003R8xlVQAR,ismael.savadogo@colorofchange.org,cfo,,corp,ams,united states,11.0
2,0030W00003qpL0bQAE,sue.nyoni@colorofchange.org,director of operations,,corp,ams,united states,11.0
3,0030W00003KBeetQAD,keith@colorofchange.org,chief of staff,"executive (ceo, cmo, coo, cfo)",corp,ams,united states,11.0
4,0030W00003KBjJgQAL,acarter@homesavings.com,avp of information technology,it: vp or director,corp,ams,united states,500.0
...,...,...,...,...,...,...,...,...
1985884,0036T00004AVmlrQAD,michael.peebles@qantasloyalty.com,secops engineer,engineer,ent,apj,australia and new zealand,250.0
1985885,0036T00004AUnJcQAL,karl.hammes@rwc.com,"director, information technology (americas)",it: vp or director,corp,ams,united states,2257.0
1985886,0036T00004AUnQvQAL,anu.kalia@rwc.com,"director, business change & transformation (emea)",,corp,emea,uk&i,2257.0
1985887,0036T00004AUnQrQAL,lee.cowcher@rwc.com,"manager, technical help desk",,corp,emea,uk&i,2257.0


In [28]:
Clean = pd.concat([WithSegment, NoSegment], axis=0, ignore_index=False).sort_index(axis = 0).fillna('')
Clean.tail(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C
1985907,0036T00004AWSVCQA5,kpepper@nossaman.com,chief operating officer,,corp,ams,united states,
1985908,0036T00004AWXs8QAH,sgilbreath@jacksonhealthcare.com,senior network engineer,,corp,ams,united states,
1985909,0036T00004AUk9jQAD,rob.juliano@yusa-oh.com,yusa it administrator,it: systems admin,corp,ams,united states,
1985910,0036T00004AW28jQAD,achakraborty@hibbertgroup.com,senior software developer,,corp,ams,united states,
1985911,0036T00004AUNsRQAX,,computer systems administrator,it: systems admin,corp,ams,united states,


# Natural Language Processing

In [29]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
#define function for remove punctuation, tokenization, remove stopwords, and stemming
#this function will be directly used with vectorizing

import re
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [31]:
def remove_punct(text):
    text_nopunct = "".join([char for char in str(text) if char not in string.punctuation])
    return text_nopunct

Clean['TRANSLATED_TITLE'] = Clean['TRANSLATED_TITLE'].apply(lambda x: remove_punct(x))

In [32]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

Clean['TRANSLATED_TITLE'] = Clean['TRANSLATED_TITLE'].apply(lambda x: tokenize(x))

In [33]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords]
    return text
 
Clean['TRANSLATED_TITLE'] = Clean['TRANSLATED_TITLE'].apply(lambda x: remove_stopwords(x))

In [34]:
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return ' '.join(text)  #change from list format to string format

Clean['TRANSLATED_TITLE'] = Clean['TRANSLATED_TITLE'].apply(lambda x: lemmatizing(x))

In [35]:
Clean.shape

(1718858, 8)

In [36]:
Clean.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C
0,0036T0000474qgbQAA,brian.guevara@colorofchange.org,senior manager,it: manager,corp,ams,united states,11.0
1,0030W00003R8xlVQAR,ismael.savadogo@colorofchange.org,cfo,,corp,ams,united states,11.0
2,0030W00003qpL0bQAE,sue.nyoni@colorofchange.org,director operation,,corp,ams,united states,11.0
3,0030W00003KBeetQAD,keith@colorofchange.org,chief staff,"executive (ceo, cmo, coo, cfo)",corp,ams,united states,11.0
4,0030W00003KBjJgQAL,acarter@homesavings.com,avp information technology,it: vp or director,corp,ams,united states,500.0


# Head of Identity & Access Management Persona

In [37]:
IdentityAccess = Clean[Clean['TRANSLATED_TITLE'].str.contains('identity|access|access management|iam|itam')]
IdentityAccess.shape

(4501, 8)

In [38]:
IdentityAccess["Persona"]='Head of Identity & Access Management'
IdentityAccess.head(5)

/var/folders/7r/1qz24gm934z0mlg9dwwq2hpr0000gn/T/ipykernel_76112/1374050305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IdentityAccess["Persona"]='Head of Identity & Access Management'


,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
391,0036T00003yfrfjQAA,tim.hill@cioxhealth.com,director iam,,ent,ams,united states,16000.0,Head of Identity & Access Management
411,0030W00003n68lyQAA,jeff.wood@cioxhealth.com,iam security engineer,security: engineer or analyst,ent,ams,united states,16000.0,Head of Identity & Access Management
446,0030W00003UkunhQAB,smcdonald@fcci-group.com,sr identity access management analyst,it: other,corp,ams,united states,840.0,Head of Identity & Access Management
458,0030W00003n5T2WQAU,lrichards@fcci-group.com,director information security access management,security: vp or director,corp,ams,united states,840.0,Head of Identity & Access Management
574,0036T00003yfexnQAA,kathleen.peters@experian.com,senior vp head fraud identity business,it: vp or director,ent,ams,united states,6586.0,Head of Identity & Access Management


In [39]:
Clean1 = Clean[~Clean.index.isin(IdentityAccess.index)]
Clean1.shape

(1714357, 8)

In [40]:
IdentityAccess2 = Clean1[Clean1['JOB_FUNCTION__C'].str.contains('identity|access')]
IdentityAccess2["Persona"]='Head of Identity & Access Management'
IdentityAccess2.shape

/var/folders/7r/1qz24gm934z0mlg9dwwq2hpr0000gn/T/ipykernel_76112/2219569580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IdentityAccess2["Persona"]='Head of Identity & Access Management'


(1, 9)

# CFO Persona

In [41]:
Clean2 = Clean1[~Clean1.index.isin(IdentityAccess2.index)]
Clean2.shape

(1714356, 8)

In [42]:
CFO = Clean2[Clean2['TRANSLATED_TITLE'].str.contains('cfo|vp finance|chief finance|head finance|finance|financial|finance leader|finance head|vice president finance|president finance|finance vice president|finance president')]
# Do not want to include manager or analyst level
CFO = CFO[~CFO['TRANSLATED_TITLE'].str.contains('analyst|manager|specialist|intern|coordinator|junior|advisor|financial assistant|planner|consultant|marketing|market|account executive|mgr|assistant director|associate director|attorney|financial service|help desk|influencer|sales')]
CFO.shape

(18548, 8)

In [43]:
CFO["Persona"]='CFO'
CFO.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
1,0030W00003R8xlVQAR,ismael.savadogo@colorofchange.org,cfo,,corp,ams,united states,11.0,CFO
58,0030W00003PG7wrQAD,david.kinns@homeserve.com,head finance system,it: manager,mm,emea,uk&i,3766.0,CFO
69,0030W00003oDp1UQAS,nick.kasmir@homeserve.com,cfo uk home repair insurance,"executive (ceo, cmo, coo, cfo)",mm,emea,uk&i,3766.0,CFO
206,0036T0000416PAuQAM,mmeixelsperger@valvoline.com,cfo,"executive (ceo, cmo, coo, cfo)",ent,ams,united states,6000.0,CFO
281,0030W00003VWO7CQAX,tinus.grobbelaar@firesyn.com,chief financial officer,,smb,ams,united states,66.0,CFO


# Head of Cyber Security Intelligence (CTI) Persona

In [44]:
Clean3 = Clean2[~Clean2.index.isin(CFO.index)]
Clean3.shape

(1695808, 8)

In [45]:
HeadofCTI = Clean3[Clean3['TRANSLATED_TITLE'].str.contains('cyber|cyber security|cybersecurity|cybercrime|cyber crime|cyber intelligence|intelligence research|threat intelligence|security research|cyber security intelligence|cybersecurity intelligence|rsi|solution architect cloud|cloud solution architect|security solution architect|solution architect security|cti director')]
HeadofCTI = HeadofCTI[~HeadofCTI['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|planner|consultant|engineer|technician|developer|finance|advisor|marketing|market|account executive|account manager|attorney|influencer|sales|business development|business intelligence|database|security engineer|application security|system admin|network admin|system analyst|system engineer')]
HeadofCTI.shape

(22998, 8)

In [46]:
HeadofCTI["Persona"]='Head of Cyber Security Intelligence'
HeadofCTI.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
13,0030W00003Zft98QAB,chris.smith@dentons.com,infrastructure cyber security manager,security: manager,corp,apj,australia and new zealand,520.0,Head of Cyber Security Intelligence
201,0036T00003yeObFQAU,baschliman@valvoline.com,director cyber security,security: vp or director,ent,ams,united states,6000.0,Head of Cyber Security Intelligence
246,0030W00003Q2c8DQAR,sives@fsarcenter.org,director cybercrime,,corp,ams,united states,13.0,Head of Cyber Security Intelligence
247,0030W00003PIlXmQAL,rheacock@fsarcenter.org,systemic cyber risk initiative manager,it: manager,corp,ams,united states,13.0,Head of Cyber Security Intelligence
250,0030W00003n5IoXQAU,rechevarria@fsarcenter.org,managerbusiness resilience systemic risk analysis crisis response cyber exercise,compliance and risk mgmt,corp,ams,united states,13.0,Head of Cyber Security Intelligence


In [47]:
Clean4 = Clean3[~Clean3.index.isin(HeadofCTI.index)]
Clean4.shape

(1672810, 8)

In [48]:
HeadofCTI2 = Clean4[Clean4['JOB_FUNCTION__C'].str.contains('cyber|cyber security|cybersecurity|cybercrime|cyber crime|cyber intelligence|intelligence research|threat intelligence|security research|cyber security intelligence|cybersecurity intelligence|cti director')]
HeadofCTI2 = HeadofCTI2[~HeadofCTI2['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|planner|consultant|engineer|technician|developer|finance|advisor|marketing|market|account executive|account manager|attorney|influencer|sales|business development|business intelligence|database|security engineer|application security|system admin|network admin|system analyst|system engineer')]
HeadofCTI2["Persona"]='Head of Cyber Security Intelligence'
HeadofCTI2.shape

(2, 9)

# Cloud Security Architect Persona

In [49]:
Clean5 = Clean4[~Clean4.index.isin(HeadofCTI2.index)]
Clean5.shape

(1672808, 8)

In [50]:
CloudSecurityArchitect = Clean5[Clean5['TRANSLATED_TITLE'].str.contains('cloud enterprise architect|enterprise architect|azure|aws|google cloud|redshift|ibm cloud|amazon web service|cloud security architect|cyber security architect|cybersecurity architect|solution architect|cloud solution architect|security solution architect|cloud implementation|cloud admin|cloud devops|cloud devsecops|cloud ops|cloud|cyber security|cybersecurity')]
# Do not want to include 'engineer', 'application', 'developer' because that belong to DevSecOps Persona
CloudSecurityArchitect = CloudSecurityArchitect[~CloudSecurityArchitect['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|planner|consultant|engineer|technician|developer|finance|advisor|marketing|market|account executive|account manager|attorney|influencer|sales|business development|business intelligence|database|security engineer|application security|system admin|network admin|system analyst|system engineer')]
CloudSecurityArchitect.shape

(19670, 8)

In [51]:
CloudSecurityArchitect["Persona"]='Cloud Security Architect'
CloudSecurityArchitect.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
11,0030W00003X3ugmQAB,jonathon.bond@dentons.com,global solution architect,it: other,corp,apj,australia and new zealand,520.0,Cloud Security Architect
52,0030W00003PG8LYQA1,lee.smart@homeserve.com,enterprise architect,it: other,mm,emea,uk&i,3766.0,Cloud Security Architect
67,0036T0000443qpFQAQ,tally.singh@homeserve.com,head cloud,,mm,emea,uk&i,3766.0,Cloud Security Architect
80,0030W00003PG86hQAD,paul.hawkins@homeserve.com,enterprise architect,it: other,mm,emea,uk&i,3766.0,Cloud Security Architect
99,0030W00003msJprQAE,tally.johal@homeserve.com,head cloud infrastructure,it: vp or director,mm,emea,uk&i,3766.0,Cloud Security Architect


In [52]:
Clean6 = Clean5[~Clean5.index.isin(CloudSecurityArchitect.index)]
Clean6.shape

(1653138, 8)

In [53]:
CloudSecurityArchitect2 = Clean6[Clean6['JOB_FUNCTION__C'].str.contains('enterprise architect|security architect|azure|aws|google cloud|redshift|ibm cloud|amazon web service|cloud security architect|cyber security architect|cybersecurity architect|solution architect|cloud solution architect|security solution architect|cloud implementation|cloud admin|cloud devops|cloud devsecops|cloud ops|cloud|cyber security|cybersecurity')]
# Do not want to include 'engineer', 'application', 'developer' because that belong to DevSecOps Persona
CloudSecurityArchitect2 = CloudSecurityArchitect2[~CloudSecurityArchitect2['TRANSLATED_TITLE'].str.contains('engineer|developer|analyst|specialist|intern|coordinator|junior|planner|consultant|technician|finance|advisor|marketing|market|account executive|account manager|attorney|influencer|sales|business development|business intelligence|database|security engineer|application security|system admin|network admin|system analyst|system engineer|application|user')]
CloudSecurityArchitect2 = CloudSecurityArchitect2[~CloudSecurityArchitect2['JOB_FUNCTION__C'].str.contains('chief information officer|cio')]
CloudSecurityArchitect2.shape

(224, 8)

In [54]:
CloudSecurityArchitect2["Persona"]='Cloud Security Architect'
CloudSecurityArchitect2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
1860,0030W00003L1Qy0QAF,jcrombie@interac.ca,senior manager emerging technology,enterprise architecture,corp,ams,canada,300.0,Cloud Security Architect
1896,0030W00003L1R5pQAF,kself@interac.ca,director mobile product platform,cloud / mobility,corp,ams,canada,300.0,Cloud Security Architect
9313,0030W00003VTuJEQA1,aspirou@olf.com,manager information technology,"infrastructure, cloud / mobility",ent,ams,united states,268.0,Cloud Security Architect
37045,0030W00003X5h1ZQAR,antonio.rodriguez@mpsa.com,functional architect,enterprise architecture,ent,emea,southern europe,1001.0,Cloud Security Architect
37329,0030W00003Y68rVQAR,mvanleeuwen@bol.com,lead architect,enterprise architecture,corp,emea,northern europe,262.0,Cloud Security Architect


# Head of DevSecOps Persona

In [55]:
Clean7 = Clean6[~Clean6.index.isin(CloudSecurityArchitect2.index)]
Clean7.shape

(1652914, 8)

In [56]:
HeadofDevSecOps = Clean7[Clean7['TRANSLATED_TITLE'].str.contains('cloud|azure|aws|amazon web service|google cloud|redshift|ibm cloud|aws engineer|kubernetes|lead security engineer|security engineer lead|cloud architect|devsecops lead|lead devsecops|head devsecops|head devops security|head security devops|devsecops|devsecops quality control|system admin devsecops|system administrator devsecops|cloud engineer|engineer cloud|cloud user|cloud developer|developer cloud|cloud architect|cloud application|application security|application security engineer')]
HeadofDevSecOps = HeadofDevSecOps[~HeadofDevSecOps['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|planner|finance|advisor|marketing|market|account executive|account manager|attorney|influencer|sales|business development|business intelligence|database|application security|system admin|network admin|system analyst|system engineer|software engineer')]
HeadofDevSecOps.shape

(3528, 8)

In [57]:
HeadofDevSecOps["Persona"]='Head of DevSecOps'
HeadofDevSecOps.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
744,0030W00003MNW4GQAX,martin.palermo@experianinteractive.com,devsecops engineer,engineer,ent,ams,united states,6586.0,Head of DevSecOps
3707,0030W00003xEMQ5QAO,rafael.schmidt@bis.org,director cloud engineering,engineer,mm,emea,central & eastern europe,501.0,Head of DevSecOps
3810,0030W00003n70b4QAA,john.mcgill@merseyside.police.uk,oracle cloud hcm consultantfunctional lead,it: other,ent,emea,uk&i,113.0,Head of DevSecOps
4563,0036T0000447BN4QAM,lmondello@ngds.com,network engineer cloud engineer project manager,,smb,ams,united states,200.0,Head of DevSecOps
5068,0036T0000444GbIQAU,jofi.yance@telenetgroup.be,cloud unified communication product engineering,,corp,emea,northern europe,2041.0,Head of DevSecOps


In [58]:
Clean8 = Clean7[~Clean7.index.isin(HeadofDevSecOps.index)]
Clean8.shape

(1649386, 8)

In [59]:
HeadofDevSecOps2 = Clean8[Clean8['JOB_FUNCTION__C'].str.contains('cloud|azure|aws|amazon web service|google cloud|redshift|ibm cloud|aws engineer|kubernetes|lead security engineer|security engineer lead|cloud architect|devsecops lead|lead devsecops|head devsecops|head devops security|head security devops|devsecops|devsecops quality control|system admin devsecops|system administrator devsecops|cloud engineer|engineer cloud|cloud user|cloud developer|developer cloud|cloud architect|cloud application|application security')]
HeadofDevSecOps2= HeadofDevSecOps2[~HeadofDevSecOps2['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|planner|finance|advisor|marketing|market|account executive|account manager|attorney|influencer|sales|business development|business intelligence|database|application security|system admin|network admin|system analyst|system engineer|software engineer')]
HeadofDevSecOps2.shape

(7, 8)

In [60]:
HeadofDevSecOps2["Persona"]='Head of DevSecOps'
HeadofDevSecOps2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
64643,0030W00003X5hvtQAB,alexander.bocharov@ivanti.com,senior software developer architect,"infrastructure, application development, enterprise architecture, business service management / ...",corp,ams,united states,285.0,Head of DevSecOps
413111,0030W00003XlFVpQAN,matt.leach@atkinsglobal.com,head device application service,"application development, cloud / mobility",mm,emea,uk&i,16530.0,Head of DevSecOps
1054882,0030W00003X5jZcQAJ,renop@wellmark.com,citrix engineer,cloud / mobility,ent,ams,united states,2692.0,Head of DevSecOps
1154196,0030W00003xEKL8QAO,dan.boughton@navistar.com,citrix technician contractor,cloud / mobility,ent,ams,united states,12557.0,Head of DevSecOps
1246002,0030W00003X5hQcQAJ,jcoleman@nvtl.com,senior application administrator,cloud / mobility,corp,ams,united states,938.0,Head of DevSecOps


# Assign Those Special Persona

In [61]:
SpecialPersona = pd.concat([IdentityAccess, IdentityAccess2, CFO, HeadofCTI, HeadofCTI2, 
                            CloudSecurityArchitect, CloudSecurityArchitect2, 
                            HeadofDevSecOps, HeadofDevSecOps2], 
                           axis=0, ignore_index=False).sort_index(axis = 0).fillna('')
SpecialPersona.to_csv('SpecialPersona.csv', index=False)

In [62]:
SpecialPersona.shape

(69479, 9)

# Exclude Those Special Personas, What is Left is for Splitting by Segment

In [63]:
Clean9 = Clean8[~Clean8.index.isin(SpecialPersona.index)]
Clean9.shape

(1649379, 8)

# Split by Company Segment

In [64]:
SMB = Clean9[Clean9['OWNER_SEGMENT__C'] == 'smb']
SMB.shape

(452271, 8)

In [65]:
CorporateMM = Clean9[Clean9['OWNER_SEGMENT__C'].isin(['corp','mm'])]
CorporateMM.shape

(605977, 8)

# SMB

In [67]:
SMB['TRANSLATED_TITLE']= SMB["TRANSLATED_TITLE"].astype(str)

/var/folders/7r/1qz24gm934z0mlg9dwwq2hpr0000gn/T/ipykernel_76112/1793582791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SMB['TRANSLATED_TITLE']= SMB["TRANSLATED_TITLE"].astype(str)


In [68]:
# Find CISO in SMB based on Translated Title
SMBCISO = SMB[SMB['TRANSLATED_TITLE'].str.contains('chief information security officer|ciso|cso|chief security officer|vp security|vpsecurity|vpinformation security|vpinfo security|vice president security|security vice president|security vp|senior director security|security senior director|head security|security head|lead security|security lead|security leader|leader security|information security|information technology security|vp information security|vp technology security|vp infrastructure security|security officer|officer security|chief information securiy officer')]
SMBCISO = SMBCISO[~SMBCISO['TRANSLATED_TITLE'].str.contains('analyst|specialist|manager|intern|coordinator|junior|advisor|finance assistant|planner|consultant|marketing|market|account executive|account manager|mgr|assistant director|associatedirector|attorney|financial service|assistant manager|associate manager|help desk|influencer|sales|accounting|business development|business intelligence|database|technician|auditor|director|dir|consultant|administrator|engineer|system admin|systems admin|network admin|admin')]

SMBCISO = SMBCISO[~SMBCISO['TRANSLATED_TITLE'].str.contains('security consultant|security architect|application security engineer|security engineer|application security|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer|it security analyst|information technology security analyst|security specialist|security system admin|security system analyst|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst')]
                                                            
SMBCISO['Persona']='Chief Information Security Officer'
SMBCISO.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
268,003d000003GUCovAAH,cpierson@viewpost.com,ciso,"executive (cio, cto, ciso)",smb,ams,united states,73.0,Chief Information Security Officer
1163,0030W00003KCkQBQA1,gerd.ellsmore@jameshay.co.uk,ciso,"executive (cio, cto, ciso)",smb,emea,uk&i,245.0,Chief Information Security Officer
1282,0030W00003UmhDPQAZ,kai.nothdurft@allianz.de,information security officer,security: other,smb,emea,central & eastern europe,150.0,Chief Information Security Officer
1324,0030W00003UmfvyQAB,jerome.herveau@metafinanz.de,information security expert,security: other,smb,emea,central & eastern europe,150.0,Chief Information Security Officer
1341,0030W00003t4SfCQAU,r.bartels@kirkpatrickprice.com,vice president security service,security: vp or director,smb,ams,united states,75.0,Chief Information Security Officer


In [69]:
SMBCISO.shape

(6941, 9)

In [70]:
RestofSMB = SMB[~SMB.index.isin(SMBCISO.index)]
RestofSMB.shape

(445330, 8)

In [71]:
# Find CISO in SMB based on JOB_FUNCTION__C
SMBCISO2 = RestofSMB[RestofSMB['JOB_FUNCTION__C'].str.contains('chief information security officer|ciso|cso|chief security officer|vp security|vpsecurity|vpinformation security|vpinfo security|vice president security|security vice president|security vp|senior director security|security senior director|head security|security head|lead security|security lead|security leader|leader security|information security|information technology security|vp information security|vp technology security|vp infrastructure security|security officer|officer security')]
SMBCISO2 = SMBCISO2[~SMBCISO2['TRANSLATED_TITLE'].str.contains('cto|chief technology|chief technical|chief technologist|cio|chief information officer|analyst|specialist|manager|intern|coordinator|junior|advisor|finance assistant|planner|consultant|marketing|market|account executive|account manager|mgr|assistant director|associatedirector|attorney|financial service|assistant manager|associate manager|help desk|influencer|sales|accounting|business development|business intelligence|database|technician|auditor|director|dir|consultant|administrator|engineer|system admin|systems admin|network admin|admin')]
SMBCISO2 = SMBCISO2[~SMBCISO2['JOB_FUNCTION__C'].str.contains('cto|advisor|technician|auditor|accounting|sales|account executive|planner|analyst|specialist|coordinator|junior|manager|support|bi|associate director|assistant director|director|dir|consultant|administrator|engineer|intern|consultant|system admin|systems admin|admin')]

SMBCISO2 = SMBCISO2[~SMBCISO2['TRANSLATED_TITLE'].str.contains('security consultant|security architect|application security engineer|security engineer|application security|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer|it security analyst|information technology security analyst|security specialist|security system admin|security system analyst|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst')]

SMBCISO2['Persona']='Chief Information Security Officer'
SMBCISO2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
46822,0030W00003fKZuLQAW,antonio.esteves@compta.pt,security business developer,information security,smb,emea,southern europe,201.0,Chief Information Security Officer
104113,0030W00003eJ9UNQA0,victor.barahona@uam.es,responsable de seguridad,information security,smb,emea,southern europe,50.0,Chief Information Security Officer
291407,0030W00003b1MEqQAM,kvilk@brfinc.com,president,"information security, it operations",smb,ams,united states,50.0,Chief Information Security Officer
301700,0030W00003n6er7QAA,novabbq@gmail.com,issm,information security,smb,ams,united states,200.0,Chief Information Security Officer
312541,0030W00003UkwZLQAZ,jwescott@wellesleybank.com,senior vp information technology risk office,"chief information officer, information security, information technology, risk, legal",smb,ams,united states,79.0,Chief Information Security Officer


In [72]:
SMBCISO2.shape

(11, 9)

In [73]:
RestofSMB2 = RestofSMB[~RestofSMB.index.isin(SMBCISO2.index)]
RestofSMB2.shape

(445319, 8)

In [74]:
# Find Head of Security in SMB based on Translated Title
SMBHeadofSecurity = RestofSMB2[RestofSMB2['TRANSLATED_TITLE'].str.contains('information security|head security|security head|head security|vp security|security vp|security director|director security|security svp|svp security|security officer|security|seguridad|isp|security manager')]
SMBHeadofSecurity = SMBHeadofSecurity[~SMBHeadofSecurity['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|finance assistant|planner|marketing|market|account executive|account manager|attorney|financial service|help desk|influencer|sales|accounting|business development|business intelligence|database|technician|auditor|consultant|administrator|engineer|system admin|systems admin|network admin|admin|engineer|technician|developer')]
SMBHeadofSecurity = SMBHeadofSecurity[~SMBHeadofSecurity['JOB_FUNCTION__C'].str.contains('analyst|specialist|intern|coordinator|junior|finance assistant|planner|marketing|market|account executive|account manager|attorney|financial service|help desk|influencer|sales|accounting|business development|business intelligence|database|technician|auditor|consultant|administrator|engineer|system admin|systems admin|network admin|admin|engineer|technician|developer')]
# I did not exclude 'manager' because SMB might have manager titled people that can be decision makers

SMBHeadofSecurity = SMBHeadofSecurity[~SMBHeadofSecurity['TRANSLATED_TITLE'].str.contains('application development|security consultant|security architect|application security engineer|security engineer|application security|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer|it security analyst|information technology security analyst|security specialist|security system admin|security system analyst|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst')]

SMBHeadofSecurity['Persona']='Head of Security'
SMBHeadofSecurity.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
263,0030W00003MOfalQAD,rrodrigue@viewpost.com,director security operation,security: vp or director,smb,ams,united states,73.0,Head of Security
380,0030W00003fJF7FQAW,jstewart@leaderdog.org,security,security: other,smb,ams,united states,157.0,Head of Security
477,0030W00003kaEBtQAM,bparathan@nps.org.au,security advisor,security: other,smb,apj,australia and new zealand,206.0,Head of Security
483,0030W00003X4SDcQAN,t.glover@npsmedicine.com.au,information security manager,,smb,,united states,206.0,Head of Security
489,003d000003GTic6AAD,charles.smart@gmsectec.com,senior manager security risk compliance,security: manager,smb,ams,united states,20.0,Head of Security


In [75]:
SMBHeadofSecurity.shape

(9997, 9)

In [76]:
RestofSMB3 = RestofSMB2[~RestofSMB2.index.isin(SMBHeadofSecurity.index)]
RestofSMB3.shape

(435322, 8)

In [77]:
# Find Head of Security based on Job_Function__C
SMBHeadofSecurity2 = RestofSMB3[RestofSMB3['JOB_FUNCTION__C'].str.contains('security:|security: manager|information security|security|security: vp or director|security: other')]
SMBHeadofSecurity2 = SMBHeadofSecurity2[~SMBHeadofSecurity2['TRANSLATED_TITLE'].str.contains('analyst|specialist|intern|coordinator|junior|finance assistant|planner|marketing|market|account executive|account manager|attorney|financial service|help desk|influencer|sales|accounting|business development|business intelligence|database|technician|auditor|consultant|administrator|engineer|system admin|systems admin|network admin|admin|engineer|technician|developer')]
SMBHeadofSecurity2 = SMBHeadofSecurity2[~SMBHeadofSecurity2['JOB_FUNCTION__C'].str.contains('security: engineer or analyst|admin')]

SMBHeadofSecurity2 = SMBHeadofSecurity2[~SMBHeadofSecurity2['TRANSLATED_TITLE'].str.contains('application development|security consultant|security architect|application security engineer|security engineer|application security|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer|it security analyst|information technology security analyst|security specialist|security system admin|security system analyst|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst')]

SMBHeadofSecurity2['Persona']='Head of Security'
SMBHeadofSecurity2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
189,0030W00003KBqxoQAD,darryn.brownfield@redspam.com,manager,security: manager,smb,emea,uk&i,196.0,Head of Security
495,003d000003GTie7AAD,gberrios@gmsectec.com,director incident response breach readiness,security: vp or director,smb,ams,united states,20.0,Head of Security
1553,0030W00003KD4LZQA1,dmccullough@sugarbush.com,director,security: director / sr. director,smb,ams,united states,165.0,Head of Security
1673,0030W00003KDyp6QAD,ejoseph@taphandles.com,director,security: director / sr. director,smb,ams,united states,100.0,Head of Security
3219,0030W00003gzJ2EQAU,tboldiston@shelde.com,account director,security: manager,smb,apj,australia and new zealand,98.0,Head of Security


In [78]:
SMBHeadofSecurity2.shape

(2304, 9)

In [79]:
RestofSMB4 = RestofSMB3[~RestofSMB3.index.isin(SMBHeadofSecurity2.index)]
RestofSMB4.shape

(433018, 8)

In [110]:
SMB = pd.concat([SMBCISO, SMBCISO2, 
                 SMBHeadofSecurity, SMBHeadofSecurity2,
                 RestofSMB4], 
                axis=0, ignore_index=False).sort_index(axis = 0).fillna('')

In [111]:
SMB.to_csv('SMB.csv',index=False)

# Corporate

In [112]:
CorporateMM['TRANSLATED_TITLE']= CorporateMM["TRANSLATED_TITLE"].astype(str)

/var/folders/7r/1qz24gm934z0mlg9dwwq2hpr0000gn/T/ipykernel_76112/4279189781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CorporateMM['TRANSLATED_TITLE']= CorporateMM["TRANSLATED_TITLE"].astype(str)


In [113]:
# Find CISO in CorporateMM based on Translated Title
CorpCISO = CorporateMM[CorporateMM['TRANSLATED_TITLE'].str.contains('chief information security officer|ciso|cso|chief security officer|vp security|vpsecurity|vpinformation security|vpinfo security|vice president security|security vice president|security vp|senior director security|security senior director|head security|security head|lead security|security lead|security leader|leader security|information security|information technology security|vp information security|vp technology security|vp infrastructure security')]
CorpCISO = CorpCISO[~CorpCISO['TRANSLATED_TITLE'].str.contains('business development|product|marketing|advisor|technician|auditor|accounting|sale|account executive|analyst|specialist|coordinator|junior|manager|support|bi|accounting|associate director|assistant director|director|dir|consultant|engineer|intern|consultant')]

CorpCISO = CorpCISO[~CorpCISO['TRANSLATED_TITLE'].str.contains('software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpCISO['Persona']='Chief Information Security Officer'
CorpCISO.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
9,0030W00003KBjJfQAL,mstricklin@homesavings.com,vice president information security officer,security: vp or director,corp,ams,united states,500.0,Chief Information Security Officer
17,0036T00004181DXQAY,chris.mctaggart@dentons.com,chief information security officer,"executive (cio, cto, ciso)",corp,,united states,520.0,Chief Information Security Officer
28,0030W00003PH2G6QAL,steve.pritchard@homeserveplc.com,group ciso,"executive (cio, cto, ciso)",mm,emea,uk&i,3766.0,Chief Information Security Officer
36,0030W00003msJpRQAU,matthew.mundry@homeserve.com,information security officer cissp cism,it: manager,mm,emea,uk&i,3766.0,Chief Information Security Officer
46,0030W00003PG6cSQAT,eleanor.smith@homeserve.com,head information security,security: other,mm,emea,uk&i,3766.0,Chief Information Security Officer


In [114]:
CorpCISO.shape

(12220, 9)

In [115]:
RestofCorp = CorporateMM[~CorporateMM.index.isin(CorpCISO.index)]
RestofCorp.shape

(593757, 8)

In [116]:
# Find CISO in CorporateMM based on Job_Function__C
CorpCISO2 = RestofCorp[RestofCorp['JOB_FUNCTION__C'].str.contains('it security|information security')]
CorpCISO2 = CorpCISO2[~CorpCISO2['TRANSLATED_TITLE'].str.contains('cio|chief information officer|chief technical officer|cto|chief technology officer|analyst|manager|junior|coordinator|specialist|engineer|infrastructure|developer|manager|information technology|sales|account executive|administrator|intern|consultant|system admin|systems admin|accounting')]

CorpCISO2 = CorpCISO2[~CorpCISO2['TRANSLATED_TITLE'].str.contains('software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpCISO2 = CorpCISO2[CorpCISO2['TRANSLATED_TITLE'].str.contains('chief|officer|vp|president|vice president|senior director|sr director|sr dir|sr director')]
CorpCISO2['Persona']='Chief Information Security Officer'
CorpCISO2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
6677,0030W00003gzePTQAY,lstokes@regionalmanagement.com,vp network risk operation,"information security, infrastructure, networking",mm,ams,united states,1638.0,Chief Information Security Officer
45165,0030W00003eKSacQAG,cachiak@centralbankmalta.org,security officer,information security,corp,emea,southern europe,330.0,Chief Information Security Officer
114899,0030W00003SAMWxQAP,ferdinand.glatzl@bawagpsk.com,security officer,information security,mm,emea,central & eastern europe,4132.0,Chief Information Security Officer
123565,0030W00003bixyzQAA,,data protection officer,"information security, it audit / it compliance",mm,emea,central & eastern europe,7067.0,Chief Information Security Officer
131626,0030W00003e0zSIQAY,gerd_fauser@hugoboss.com,data protection officer,information security,mm,ams,united states,14633.0,Chief Information Security Officer


In [117]:
CorpCISO2.shape

(24, 9)

In [118]:
RestofCorp2 = RestofCorp[~RestofCorp.index.isin(CorpCISO2.index)]
RestofCorp2.shape

(593733, 8)

In [119]:
# Find Head of Security Operations based on Translated Title
CorpHeadofSecurityOperations = RestofCorp2[RestofCorp2['TRANSLATED_TITLE'].str.contains('security|information security|SOC|security operation center|security operation|information technology security|architect security|architecture security|security leader|security lead|head security|lead security|security head|security director|director security|director soc|soc manager|manager soc|principal security|security principal|director infosec|infrastructure')]
CorpHeadofSecurityOperations = CorpHeadofSecurityOperations[~CorpHeadofSecurityOperations['TRANSLATED_TITLE'].str.contains('project manager|engineer|technician|accounting|sales|account executive|analyst|specialist|coordinator|junior|support|bi|accounting|consultant|engineer|intern|system admin|systems admin|administrator')]
CorpHeadofSecurityOperations = CorpHeadofSecurityOperations[~CorpHeadofSecurityOperations['JOB_FUNCTION__C'].str.contains('it: manager|operation|accounting|sales|account executive|analyst|specialist|coordinator|junior|support|bi|accounting|consultant|intern|engineer|system admin|systems admin')]

CorpHeadofSecurityOperations = CorpHeadofSecurityOperations[~CorpHeadofSecurityOperations['TRANSLATED_TITLE'].str.contains('application development|software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpHeadofSecurityOperations['Persona']='Head of Security Operations'
CorpHeadofSecurityOperations.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
8,0030W00003KBjJhQAL,thoward@homesavings.com,assistant vp manager network security,security: vp or director,corp,ams,united states,500.0,Head of Security Operations
15,0030W00003VUUz0QAH,matthew.phillips@dentons.com,security officer,security: other,corp,apj,australia and new zealand,520.0,Head of Security Operations
44,0030W00003lNBZ7QAO,bobby.khatkar@homeserve.com,head infrastructure delivery,,mm,emea,uk&i,3766.0,Head of Security Operations
165,0030W00003qovGvQAI,sarah.abdelkader@keoliscs.com,director security safety compliance audit,security: vp or director,corp,ams,united states,2400.0,Head of Security Operations
290,0036T0000408sOkQAI,alex.fisher@fisherfoods.com,director social network marketing,marketing,corp,ams,united states,251.0,Head of Security Operations


In [120]:
CorpHeadofSecurityOperations.shape

(32310, 9)

In [121]:
RestofCorp3 = RestofCorp2[~RestofCorp2.index.isin(CorpHeadofSecurityOperations.index)]
RestofCorp3.shape

(561423, 8)

In [122]:
# Find Head of Security Operations based on Job_Function__C
CorpHeadofSecurityOperations2 = RestofCorp3[RestofCorp3['JOB_FUNCTION__C'].str.contains('security|information security|security:|security: vp or director|security: director|director infosec|info sec')]
CorpHeadofSecurityOperations2 = CorpHeadofSecurityOperations2[~CorpHeadofSecurityOperations2['TRANSLATED_TITLE'].isin(['cmo','ceo','coo','cto'])]
CorpHeadofSecurityOperations2 = CorpHeadofSecurityOperations2[~CorpHeadofSecurityOperations2['TRANSLATED_TITLE'].str.contains('analyst|engineer|technician|cio|chief information officer|chief technology officer|accounting|sales|account executive|junior|coordinator|specialist|engineer|infrastructure|developer|information technology|sales|account executive|left company|consultant|system admin|systems admin')]
CorpHeadofSecurityOperations2 = CorpHeadofSecurityOperations2[~CorpHeadofSecurityOperations2['JOB_FUNCTION__C'].str.contains('analyst|engineer')]
CorpHeadofSecurityOperations2 = CorpHeadofSecurityOperations2[~CorpHeadofSecurityOperations2['TRANSLATED_TITLE'].str.contains('application development|software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpHeadofSecurityOperations2['Persona']='Head of Security Operations'
CorpHeadofSecurityOperations2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
108,0030W00003KBkRcQAL,val.penn@etelimited.co.uk,group data protection officer,security: other,mm,emea,uk&i,171.0,Head of Security Operations
117,0030W00003KBkWwQAL,darren.topham@btgplc.com,group head,security: vp or director,corp,emea,uk&i,1631.0,Head of Security Operations
256,0030W00003kaZ9tQAE,mflemings@fsarcenter.org,issm,security: manager,corp,ams,united states,13.0,Head of Security Operations
258,0030W00003X6Aq4QAF,fbicaldo@fsarcenter.org,information security project manager,security: manager,corp,ams,united states,13.0,Head of Security Operations
1123,0030W00003bj6SVQAY,eleanor.foster@provident.co.uk,senior info sec manager,security: manager,mm,emea,uk&i,4865.0,Head of Security Operations


In [123]:
CorpHeadofSecurityOperations2.shape

(5558, 9)

In [124]:
RestofCorp4 = RestofCorp3[~RestofCorp3.index.isin(CorpHeadofSecurityOperations2.index)]
RestofCorp4.shape

(555865, 8)

In [125]:
# Find CIO in based on Translated Title
CorpCIO = RestofCorp4[RestofCorp4['TRANSLATED_TITLE'].str.contains('chief information officer|cio|information technology|head it|it head|president it|president it|it vp|vp it')]
CorpCIO = CorpCIO[~CorpCIO['TRANSLATED_TITLE'].str.contains('technician|help desk|facility|influencer|administrator|accounting|sales|account executive|analyst|specialist|coordinator|junior|manager|support|bi|accounting|associate director|assistant director|director|dir|consultant|engineer|intern|consultant|system admin|systems admin')]

CorpCIO = CorpCIO[~CorpCIO['TRANSLATED_TITLE'].str.contains('software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpCIO['Persona']='Chief Information Officer'
CorpCIO.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
4,0030W00003KBjJgQAL,acarter@homesavings.com,avp information technology,it: vp or director,corp,ams,united states,500.0,Chief Information Officer
12,0030W00003UkBoKQAV,peter.fitzgibbon@dentons.com,cio,"executive (cio, cto, ciso)",corp,apj,australia and new zealand,520.0,Chief Information Officer
35,0030W00003azJtaQAE,femi.bamisaiye@homeserve.com,chief information officer uk,"executive (cio, cto, ciso)",mm,emea,uk&i,3766.0,Chief Information Officer
113,0030W00003TEuwmQAD,alex.rammal@btgplc.com,cio,"executive (cio, cto, ciso)",corp,emea,uk&i,1631.0,Chief Information Officer
255,0030W00003MOHaMQAX,wgreene@arcgroup.org,chief information officer chief securirty officer,"executive (cio, cto, ciso)",corp,ams,united states,13.0,Chief Information Officer


In [126]:
CorpCIO.shape

(30603, 9)

In [127]:
RestofCorp5 = RestofCorp4[~RestofCorp4.index.isin(CorpCIO.index)]
RestofCorp5.shape

(525262, 8)

In [128]:
# Find CIO based on Job_Function__C
CorpCIO2 = RestofCorp5[RestofCorp5['JOB_FUNCTION__C'].str.contains('cio|chief information officer|information technology|it: vp or director')]
CorpCIO2 = CorpCIO2[~CorpCIO2['TRANSLATED_TITLE'].str.contains('head operation|operational support|vice president technology|technician|help desk|facility|influencer|administrator|director|cto|chief technology officer|analyst|accounting|account executive|manager|junior|coordinator|specialist|engineer|infrastructure|developer|manager|information technology|sales|account executive|consultant')]

CorpCIO2 = CorpCIO2[~CorpCIO2['TRANSLATED_TITLE'].str.contains('software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpCIO2['Persona']='Chief Information Officer'
CorpCIO2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
244,0030W00003KBwxPQAT,raymusser@gmail.com,executive,"executive (cio, cto, ciso)",corp,ams,united states,13.0,Chief Information Officer
472,0036T000042fIdAQAU,dpatel2@fcci-group.com,vice president,it: vp or director,corp,ams,united states,840.0,Chief Information Officer
1367,0030W00003PHzQAQA1,donald.kneisel@seminolehardrock.com,vp,it: vp or director,corp,ams,united states,2000.0,Chief Information Officer
1842,003d000003GSCQ3AAP,vsilvestri@evertz.com,vice president software system,it: vp or director,corp,ams,canada,20.0,Chief Information Officer
1895,0030W00003UmVqGQAV,rkohli@interac.ca,vice president operation technology,it: vp or director,corp,ams,united states,300.0,Chief Information Officer


In [129]:
CorpCIO2.shape

(9250, 9)

In [130]:
RestofCorp6 = RestofCorp5[~RestofCorp5.index.isin(CorpCIO2.index)]
RestofCorp6.shape

(516012, 8)

In [131]:
# Find Head of IT Operations (VP/Director) based on Translated Title
CorpHeadofITVP = RestofCorp6[RestofCorp6['TRANSLATED_TITLE'].str.contains('information technology|information system|it director|director it|president it|president it|vp it|vp it|it infrastructure|it operation|it engineer|it service')]
CorpHeadofITVP = CorpHeadofITVP[~CorpHeadofITVP['TRANSLATED_TITLE'].str.contains('manager|mgr|head|lead|audit manager|technician|analyst|accounting|account executive|sales|specialist|coordinator|junior|intern|deposit|consultant|system admin|systems admin')]

CorpHeadofITVP = CorpHeadofITVP[~CorpHeadofITVP['TRANSLATED_TITLE'].str.contains('application development|software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpHeadofITVP['Persona']='Head of IT Operations (VP/Director)'
CorpHeadofITVP.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
34,0036T000040BvTMQA0,alison.moses@homeserve.com,director business information technology,it: vp or director,mm,emea,uk&i,3766.0,Head of IT Operations (VP/Director)
42,0036T0000417udrQAA,lisa.platt@homeserve.com,director information technology,it: vp or director,mm,,united states,3766.0,Head of IT Operations (VP/Director)
98,0036T000040BviCQAS,priya.rajagopalan@homeserve.com,digital development director information technology,it: vp or director,mm,emea,uk&i,3766.0,Head of IT Operations (VP/Director)
100,0030W00003PG889QAD,richard.lockwood@homeserve.com,director information technology,it: vp or director,mm,emea,uk&i,3766.0,Head of IT Operations (VP/Director)
280,0030W00003VWOTgQAP,tonyw@juice4u.com,director information technology,it: vp or director,corp,ams,united states,500.0,Head of IT Operations (VP/Director)


In [132]:
CorpHeadofITVP.shape

(26042, 9)

In [133]:
RestofCorp7 = RestofCorp6[~RestofCorp6.index.isin(CorpHeadofITVP.index)]
RestofCorp7.shape

(489970, 8)

In [134]:
# Find Head of IT Operations (VP/Director) based on Job_Function__C
CorpHeadofITVP2 = RestofCorp7[RestofCorp7['JOB_FUNCTION__C'].str.contains('information technology|information system|it director|director it|president it|president it|vp it|vp it|it infrastructure|it operation|it engineer|it service|it: vp or director|it-infrastructure')]
CorpHeadofITVP2 = CorpHeadofITVP2[~CorpHeadofITVP2['TRANSLATED_TITLE'].str.contains('manager|mgr|lead|head|audit manager|technician|analyst|specialist|coordinator|junior|intern|deposit|engineer|infrastructure|developer|information technology|sales|account executive|consultant|system admin|systems admin|digital|strategy|product|marketing')]

CorpHeadofITVP2 = CorpHeadofITVP2[~CorpHeadofITVP2['TRANSLATED_TITLE'].str.contains('application development|software developer|software engineer|security consultant|security architect|application security engineer|application security|security engineer|security system analyst|security system admin|network security analyst|security system technician|soc analyst|security operation center analyst|information security analyst|security analyst|it admin|information technology admin|it security analyst|information technology security analyst|security specialist|system admin|admin|system analyst|it engineer|information technology engineer|support specialist|system engineer')]

CorpHeadofITVP2['Persona']='Head of IT Operations (VP/Director)'
CorpHeadofITVP2.head(5)

,ID,EMAIL,TRANSLATED_TITLE,JOB_FUNCTION__C,OWNER_SEGMENT__C,GEO_LOCATION_LEVEL1__C,REGION__C,EMPLOYEE_COUNT__C,Persona
39,0030W00003PG6bsQAD,ed.foden@homeserve.com,director technology,it: vp or director,mm,emea,uk&i,3766.0,Head of IT Operations (VP/Director)
83,0030W00003PG7vfQAD,graham.moore@homeserve.com,director service management,it: vp or director,mm,emea,uk&i,3766.0,Head of IT Operations (VP/Director)
104,0030W00003Q2x4ZQAR,david.willock@welcomebreak.co.uk,group director,it: vp or director,mm,ams,united states,171.0,Head of IT Operations (VP/Director)
168,0030W00003oQ5OKQA0,bryan.gobiel@keoliscs.com,director d operation,it: vp or director,corp,ams,united states,2400.0,Head of IT Operations (VP/Director)
890,0030W00003KCM8fQAH,tonydimaio@desktopmetal.com,director,it-infrastructure,corp,ams,united states,203.0,Head of IT Operations (VP/Director)


In [135]:
CorpHeadofITVP2.shape

(25304, 9)

In [136]:
RestofCorp8 = RestofCorp7[~RestofCorp7.index.isin(CorpHeadofITVP2.index)]
RestofCorp8.shape

(464666, 8)

In [167]:
Corp = pd.concat([CorpCISO, CorpCISO2, 
                  CorpHeadofSecurityOperations, CorpHeadofSecurityOperations2, 
                  CorpCIO, CorpCIO2, 
                  CorpHeadofITVP, CorpHeadofITVP2,
                  RestofCorp8], axis=0, ignore_index=False).sort_index(axis = 0).fillna('')
Corp.to_csv('Corporate.csv',index=False)